In [5]:
import os
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from openai import OpenAI

# 환경 변수 로드
load_dotenv()

# OpenAI 초기화
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()

# PDF 로드 및 텍스트 분할
def load_and_split_pdf(pdf_path, chunk_size=1000, chunk_overlap=100):
    """
    PDF 문서를 로드하고 텍스트를 분할합니다.
    """
    try:
        loader = PDFPlumberLoader(pdf_path)
        docs = loader.load()
        print(f"✅ PDF에서 로드된 문서: {docs[:2]}...")  # 첫 두 개 문서만 출력
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_docs = text_splitter.split_documents(docs)
        print(f"✅ 분할된 문서 개수: {len(split_docs)}")
        return split_docs
    except Exception as e:
        raise ValueError(f"PDF 로드 및 분할 중 오류가 발생했습니다: {str(e)}")

# 임베딩 모델 생성
def create_embeddings():
    """
    OpenAI 임베딩 모델 생성
    """
    try:
        embeddings = OpenAIEmbeddings()
        print("✅ 임베딩 모델 생성 완료")
        return embeddings
    except Exception as e:
        raise ValueError(f"임베딩 모델 생성 중 오류가 발생했습니다: {str(e)}")

# 벡터 저장소 생성
def create_vector_store(documents, embeddings):
    """
    텍스트를 벡터로 변환하고 검색 가능한 벡터 저장소를 생성합니다.
    """
    try:
        vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)
        print(f"✅ 생성된 벡터 저장소: {vectorstore}")
        return vectorstore
    except Exception as e:
        raise ValueError(f"벡터 저장소 생성 중 오류가 발생했습니다: {str(e)}")

# RAG 체인 생성
def create_rag_chain(vectorstore):
    """
    검색 가능한 RAG 체인을 생성합니다.
    """
    try:
        # 1. 검색기 생성
        retriever = vectorstore.as_retriever()
        print(f"✅ 생성된 검색기: {retriever}")

        # 2. 프롬프트 정의
        prompt = PromptTemplate.from_template(
            """너는 주어진 데이터를 활용해서 단어의 정의를 알려주는 용어사전 AI야. +
            다음 검색된 context를 사용해서 question에 대답해줘.
            답을 모르면, '알 수 없습니다.'라고 대답해.

            # Context : {context}
            # Question : {question}
            # Answer :
            """
        )
        print(f"✅ 생성된 프롬프트 템플릿: {prompt}")

        # 3. 언어 모델 정의
        llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
        print("✅ 언어 모델 생성 완료")

        # 4. 체인 생성
        rag_chain = (
            {'context': retriever, 'question': RunnablePassthrough()}  # 질문과 검색 문맥 연결
            | prompt  # 검색된 문맥과 질문을 기반으로 프롬프트 생성
            | llm  # 언어 모델로 답변 생성
            | StrOutputParser()  # 답변을 텍스트로 변환
        )
        print("✅ RAG 체인 생성 완료")
        return rag_chain
    except Exception as e:
        raise ValueError(f"RAG 체인 생성 중 오류가 발생했습니다: {str(e)}")

# 질문에 대한 답변 생성
def generate_rag_answer(question, rag_chain):
    """
    질문을 입력받아 RAG 체인을 통해 답변을 생성합니다.
    """
    try:
        print(f"✅ 입력된 질문: {question}")
        
        # 입력 데이터 출력 및 타입 확인
        input_data = question
        print(f"✅ 입력 데이터: {input_data}")
        print(f"✅ 입력 데이터 타입: {type(input_data)}")
        
        # RAG 체인에 입력
        answer = rag_chain.invoke(input_data)
        print(f"✅ 생성된 답변: {answer}")
        return answer
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"


# PDF 문서 기반 RAG 시스템 초기화
def initialize_rag_system(pdf_path):
    """
    PDF 문서를 기반으로 RAG 시스템 초기화.
    """
    try:
        print("📄 PDF 파일 로드 및 텍스트 분할 중...")
        documents = load_and_split_pdf(pdf_path)

        print("📊 임베딩 및 벡터 저장소 생성 중...")
        embeddings = create_embeddings()
        vectorstore = create_vector_store(documents, embeddings)

        print("🔗 RAG 체인 생성 중...")
        rag_chain = create_rag_chain(vectorstore)

        return rag_chain
    except Exception as e:
        raise ValueError(f"RAG 시스템 초기화 중 오류가 발생했습니다: {str(e)}")

# 테스트 실행
if __name__ == "__main__":
    # PDF 파일 경로
    pdf_path = "C:\RepoEx\AI_Python_AssistantBot\data\converted_data_with_metadata.pdf"
    
    try:
        # RAG 시스템 초기화
        rag_chain = initialize_rag_system(pdf_path)
        
        print("✅ RAG 시스템이 성공적으로 초기화되었습니다.")

        # 테스트 질문 입력
        question = "파이썬의 for문의 정의를 알려줘"
        print(f"질문: {question}")
        
        # 답변 생성
        answer = generate_rag_answer(question, rag_chain)
        print(f"답변: {answer}")
    except Exception as e:
        print(f"❌ 테스트 실행 중 오류가 발생했습니다: {str(e)}")

📄 PDF 파일 로드 및 텍스트 분할 중...
✅ PDF에서 로드된 문서: [Document(metadata={'source': 'C:\\RepoEx\\AI_Python_AssistantBot\\data\\converted_data_with_metadata.pdf', 'file_path': 'C:\\RepoEx\\AI_Python_AssistantBot\\data\\converted_data_with_metadata.pdf', 'page': 0, 'total_pages': 349, 'Producer': 'PyPDF2', 'Title': 'Jump to Phython', 'Author': 'Park', 'Subject': 'Python', 'Keywords': 'Python, Book', 'Creator': 'PyPDF2 Library'}, page_content="1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본 순간부터 지금까지 줄곧 파이썬과 함께 지내\n온 듯하다. ‘프로그래밍은 어렵고 지루하다’라는 고정관념을 가지고 있던 필자에게 파이\n썬은 커다란 충격으로 다가왔다. 여러분도 이 책을 통해 파이썬의 매력에 흠뻑 빠져 보\n기를 바란다. 01장에서는 파이썬의 특징과 장단점을 알아보고 파이썬 프로그래밍을 위한\n환경 구축 방법에 대해 배운다. 그리고 간단한 파이썬 프로그램도 작성해 본다.\n01-1 파이썬이란?\n파이썬(Python)은 1990년 암스테르담의 귀도 반 로섬(Guido van rossum)이 개발한 인터\n프리터 언어이다. 귀도는 파이썬이라는 이름을 자신이 좋아하는 코미디 쇼인 ‘몬티 파이\n썬의 날아다니는 서커스(Monty python's flying circus)’에서 따왔다고 한다.\n인터프리터 언어란 소스 코드를 한 줄씩 해석한 후 그때그때 실행해 결과를 바로 확인할\n수 있는 언어를 말한다.\n파이썬의 사전적 의미는 ‘고대 신화에 나오는 파르나소스 산의 동굴에 살던 큰 뱀’을 뜻\n하며, 아폴로 신이 